In [1]:
import pandas as pd


In [2]:
excel_path = "../data/4_GEOMAR-relevant_COG_grantees_and_panel_members_2013-2024.xlsx"
sheet_name = "Panel members"
filter_call = "CoG 2024"

In [3]:
df = pd.read_excel(excel_path, sheet_name=sheet_name)
# strip the column names
df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
display(df.head())

# filter
#df = df[df['Call'] == filter_call].reset_index(drop=True)

# Zeige alle Spaltennamen
print("Verfügbare Spalten:")
print(df.columns.tolist())



# Nur wenn die Spalten existieren
if 'First name' in df.columns and 'Last name' in df.columns:
    df['Name'] = df["First name"] + " " + df["Last name"].str.strip()
else:
    print("\n⚠️ Kann 'Name' nicht erstellen - Spalten fehlen!")

display(df.head())

,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call
0,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Member,LS,CoG 2014
1,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Chair,LS,CoG 2016
2,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2014
3,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2016
4,Ameli,Nadia,UCL Institute for Sustainable Resources,"Climate Finance. climate finance, networks and...",SH7,Member,SH,CoG 2024


Verfügbare Spalten:
['Last name', 'First name', 'Host institution (HI)', 'Field', 'Panel', 'Function', 'Domain', 'Call']


,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call,Name
0,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Member,LS,CoG 2014,Jon Agren
1,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Chair,LS,CoG 2016,Jon Agren
2,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2014,Francis Albarede
3,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2016,Francis Albarede
4,Ameli,Nadia,UCL Institute for Sustainable Resources,"Climate Finance. climate finance, networks and...",SH7,Member,SH,CoG 2024,Nadia Ameli


In [4]:
# show dataframe where Name is not null and other columnas contain Nan
df_member = df[df['Name'].notnull()].reset_index(drop=True)

empty_fields = df_member[df_member["Field"].isnull()]
display(empty_fields)

unique_members = empty_fields['Name'].unique()
print(f"\nAnzahl der einzigartigen Mitglieder ohne 'Field': {len(unique_members)}")



,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call,Name
8,Anderluh,Gregor,NaN,NaN,LS9,Member,LS,CoG 2021,Gregor Anderluh
9,Anderluh,Gregor,NaN,NaN,LS9,Member,LS,CoG 2023,Gregor Anderluh
15,Arndt,Nicholas,NaN,NaN,PE10,Member,PE,CoG 2021,Nicholas Arndt
16,Arndt,Nicholas,NaN,NaN,PE10,Member,PE,CoG 2023,Nicholas Arndt
19,Bai,Yuling,NaN,NaN,LS9,Member,LS,CoG 2021,Yuling Bai
...,...,...,...,...,...,...,...,...,...
543,Virrantaus,Kirsi,NaN,NaN,SH7,Member,SH,CoG 2021,Kirsi Virrantaus
544,Virrantaus,Kirsi,NaN,NaN,SH7,Member,SH,CoG 2023,Kirsi Virrantaus
561,Wu,Xiaogang,NaN,NaN,SH7,Member,SH,CoG 2021,Xiaogang Wu
562,Xu,Chong-Yu,NaN,NaN,PE10,Member,PE,CoG 2021,Chong-Yu Xu



Anzahl der einzigartigen Mitglieder ohne 'Field': 68


In [5]:
from pprint import pprint

pprint(unique_members)
pprint(len(unique_members))

array(['Gregor Anderluh', 'Nicholas Arndt', 'Yuling Bai',
       'Michael Benton', 'Petra Boevink', 'Peter Bols',
       'Kim A. Boutilier', 'Donald E. Canfield', 'Jørgen Carling',
       'Paul Cloke', 'Marcus Collier', 'Pierre Phillippe Combes',
       'Pierre-Philippe Combes', 'Cláudia de Freitas', 'Ben Derudder',
       'Paul Ekins', 'Martin Funk', 'Toni Gabaldón', 'Andrey Ganopolski',
       'Alexandros Gasparatos', 'Ashleigh Griffin', 'Lars-Anders Hansson',
       'Marie E. Herberstein', 'Niels Høiby', 'Robert Holt',
       'Corinna Hoose', 'Dirk Jacobs', 'Adrian Jenkins', 'Nadja Kabisch',
       'Henrik Kaessmann', 'Inger Kappel Schmidt', 'Kristine Kern',
       'Phoebe Koundouri', 'Julie LaRoche', 'Iñigo Lasa Uzcudun',
       'Philippe Lemanceau', 'Edward Lemke', 'Regina Lindborg',
       'Rosaly Lopes', 'Alessandra Magistrato', 'Johanna Mappes',
       'Anders Meibom', 'Andrzej Mizgajski', 'Andrew Mondschein',
       'Michael Obersteiner', 'Spyros Pandis', 'Dario Papale',
     

In [6]:
import ollama 
import time
import requests

def fetch_openalex_id(name:str) -> dict | None:

    search_name = name.replace(" ", "+")
    base_url = f"https://api.openalex.org/authors?filter=display_name.search:{search_name}"
    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()
        results_list = data.get('results', [])
        
        if not results_list:
            print(f"⚠️ Keine Ergebnisse für {name}")
            return None
        
        # Nimm das erste Ergebnis
        first_result = results_list[0]
        #pprint(type(first_result))
        #pprint(first_result.keys())
        
        # Extrahiere Informationen
        topics = first_result.get('topics', [])

        # 🔹 Sortieren nach 'count' (absteigend)
        sorted_topics = sorted(topics, key=lambda x: x['count'], reverse=True)

        # 🔹 Die größten 4 auswählen
        top_topics = [t['display_name'] for t in sorted_topics[:1]]

        #pprint(top_topics)
        affiliations = first_result.get('affiliations', [])
        x_concepts = first_result.get('x_concepts', [])

        topics_fields_list = [topic['display_name'] for topic in topics]
        #pprint(topics_fields_list)
        
        affiliation = affiliations[0]["institution"]["display_name"] if affiliations else None
        #pprint(affiliation)
        
        x_concepts_fields_list = [concept['display_name'] for concept in x_concepts if concept["score"] >= 90]
        #pprint(x_concepts_fields_list)
        #pprint(x_concepts)
        

        # Speichere die Ergebnisse in einem Dictionary
        results = {
            "Name": name,
            "topics": top_topics,
            "affiliation": affiliation,
            "x_concepts": x_concepts_fields_list
        }
        return results
    else:
        print(f"⚠️ Fehler bei der Anfrage: {response.status_code}")
        return None
    


def process_researcher_profile(results:dict, stream:bool = False, model:str = "deepseek-r1:1.5b") -> str:

    prompt1 = f"""
    You are a scientific summarizer. Based on the following structured information about a researcher, write a concise, natural-sounding academic profile of about 3 sentences. Focus on their main research areas, using fluent academic language similar to a bio in a research paper.
    Do not call them postdoc or similar terms; just focus on their expertise and research themes.
    Input data:
    
    {results}

    Output format:
    A short paragraph (≈3 sentences) describing the person’s expertise and research themes, written in English.
    """

    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt1}],
        stream=True,
    )

    # Antwort puffern und gleichzeitig anzeigen
    response_text = ""
    for chunk in response:
        content = chunk.get("message", {}).get("content", "")
        print(content, end="", flush=True)
        response_text += content

    # # Jetzt hast du alles gesammelt
    # print("\n\n✅ Streaming beendet.")

    return response_text


df_member_info = pd.DataFrame(columns=["name", "affiliation", "topics", "x_concepts", "generated_profile"])

for member in unique_members[:]:
    results = fetch_openalex_id(member)
    if results:
        print(f"\n\n--- Profil für {member} ---")
        response_text = process_researcher_profile(
            model="deepseek-r1:1.5b",
            results=results,
            stream=True
        )
        #print(f"\nGeneriertes Profil:\n{response_text}")
        time.sleep(2)  # Kurze Pause zwischen den Anfragen
    else:
        print(f"Keine Daten für {member} gefunden.")

    df_member_info = pd.concat([df_member_info, pd.DataFrame([{
        "name": member,
        "affiliation": results.get("affiliation") if results else None,
        "topics": results.get("topics") if results else None,
        "x_concepts": results.get("x_concepts") if results else None,
        "generated_profile": response_text if results else None
    }])], ignore_index=True)

display(df_member_info)



--- Profil für Gregor Anderluh ---
Gregor Anderluh is an expert in lipid membrane structure and behavior, with a strong focus on its role in biological systems across cell biology and biophysics. His research integrates concepts from biochemistry, genetics, and chemistry to advance our understanding of how these structures function within living organisms.

--- Profil für Nicholas Arndt ---
Institut des Sciences de la Terre is where Nicholas Arndt conducts his research. His work primarily involves the analysis of geological samples using advanced analytical techniques to identify and study Earth's dynamic processes.

--- Profil für Yuling Bai ---
Yuling Bai specializes in the study of plant-microbe interactions and plant-immune systems, employing a multidisciplinary approach that integrates biology, genetics, chemistry, and biochemistry to advance our understanding of how microbes shape plant health and resilience. Their research is at the forefront of theoretical and applied studies

,name,affiliation,topics,x_concepts,generated_profile
0,Gregor Anderluh,National Institute of Chemistry,[Lipid Membrane Structure and Behavior],"[Biology, Chemistry, Biochemistry, Genetics]",Gregor Anderluh is an expert in lipid membrane...
1,Nicholas Arndt,Institut des Sciences de la Terre,[Geological and Geochemical Analysis],[],Institut des Sciences de la Terre is where Nic...
2,Yuling Bai,Wageningen University & Research,[Plant-Microbe Interactions and Immunity],"[Biology, Genetics, Chemistry, Biochemistry]",Yuling Bai specializes in the study of plant-m...
3,Michael Benton,University of Bristol,[Evolution and Paleontology Studies],[],Michael Benton is a researcher at the Universi...
4,Petra Boevink,James Hutton Institute,[Plant-Microbe Interactions and Immunity],"[Biology, Chemistry, Genetics, Biochemistry]",Petra Boevink is a researcher at the James Hut...
...,...,...,...,...,...
63,Yusak Susilo,BOKU University,[Urban Transport and Accessibility],[],Yusak Susilo is a distinguished researcher at ...
64,Pilar Villamor,GNS Science,[earthquake and tectonic studies],"[Geology, Seismology]",Pilar Villamor specializes in understanding th...
65,Kirsi Virrantaus,Aalto University,[Geographic Information Systems Studies],[],Her research focuses on geographic information...
66,Xiaogang Wu,New York University Shanghai,[China's Socioeconomic Reforms and Governance],[],Xiaogang Wu has conducted extensive research o...


In [ ]:
from ddgs import DDGS
import requests
from bs4 import BeautifulSoup

# Suche durchführen
query = "Prof Kim A. Boutilier"
results = DDGS().text(query, region='wt-wt', max_results=5)

# Ergebnisse anzeigen
for r in results:
    print(f"\n{r['title']}\n{r['href']}")

# Textinhalt der ersten Seite scrapen
url = list(results)[0]["href"]
html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}).text
soup = BeautifulSoup(html, "html.parser")

text = ' '.join(p.get_text() for p in soup.find_all("p"))
print("\n--- Textauszug ---\n", text[:1000])



dr. KA (Kim) Boutilier - WUR
https://www.wur.nl/en/persons/kim-boutilier.htm

Kim Boutilier's research works | Wageningen University & Research ...
https://www.researchgate.net/scientific-contributions/Kim-Boutilier-20800113

Kim Boutilier - AIChE
https://www.aiche.org/sbe/community/bio/kim-boutilier

Kim Boutilier - Wageningen University & Research
https://research.wur.nl/en/persons/kim-boutilier

Kim Boutilier Inventions, Patents and Patent Applications - Justia ...
https://patents.justia.com/inventor/kim-boutilier

--- Textauszug ---
 Our Research Our research is focussed on understanding the molecular-genetic basis for plant regeneration through embryogenesis. We study different types of in vitro regeneration, including embryogenesis from haploid male and female gametophytes (pollen and embryo sac) and embryogenesis from vegetative tissues (somatic embryogenesis), as well as different types of seed-based embryogenesis systems (zygotic embryogenesis, egg cell parthenogenesis, mater

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Kim A. Boutilier")


"We found 10 Kimberley Boutilier's profiles > Get contact information, phone numbers, home addresses, age, background check, photos, and other public records [Updated: May 7, 2025]. Hi, my name is Kathleen Tracey, the couple in the picture are my long time friend Kim (Van… Kathleen Tracey needs your support for Gerard's battle with Cancer Search Boutilier family obituaries and memoriams on Legacy.com. There are 326 obituaries and memoriams for the surname Boutilier . View Profile 370 Jeremy Ln, Weatherford, TX 76088 also known as Kim A Witt Kimberlya Witt Kimberly A Witt Kim A Boutilier Kimberly A Boutilier Kimberley A Boutilier related to Carolyn Richardson, 66 James Dunn, 56 Matthew Fisher Barbara Boutilier , 80 Hazel Boutilier , 57 has lived in Weatherford, TX Kenosha, WI phone number (817) 475-6213 (817 ... 370 Jeremy Ln, Weatherford, TX 76088 Verified also known as Kimberly A Witt Kimberlya Witt Kim A Witt Kimberly A Boutilier Kim A Boutilier Kimberley A Boutilier related to Carol

In [ ]:


from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

search.invoke("Gregor Anderluh")

'September 11, 2025 - Schulz. Boutilier went to Vancouver Film School in 1996, as well as the University of King\'s College. 3 weeks ago - The EP featured appearances from Pusha T and Kurtis Blow on the songs "Strangers" and "Going Diamond" respectively, as well as song writing credits from the likes of Ryan Tedder (of OneRepublic), Grammy award-winning songwriter Evan Bogart, Cherry Cherry Boom Boom, and producer Doctor Rosen Rosen. The EP received mixed reviews, comparing Boutilier to the likes of Katy Perry, Nicki Minaj, and Gwen Stefani. July 11, 2025 - Paul André Boutilier (born May 3, 1963) is a Canadian former professional ice hockey defenceman who played with several National Hockey League teams in the 1980s. September 22, 2025 - Teahoon Park; Chihyun Park; Byeonggwan Kim; Haejin Shin; Eunkyoung Kim (2013). "Flexible PEDOT electrodes with large thermoelectric power factors to generate electricity by the touch of fingertips". Energy & Environmental Science. 6 (3): 788. doi:10.10

In [ ]:
search = DuckDuckGoSearchResults(output_format="list", num_results=5)

search.invoke("Prof Gregor Anderluh")

[{'snippet': 'Es ist bundesweit nicht einheitlich geregelt, welchen Titel ein Juniorprofessor führen soll (Jun.-Prof. oder Prof.) und ob ihm das Promotionsrecht zuerkannt wird.',
  'title': 'Professor – Wikipedia',
  'link': 'https://de.wikipedia.org/wiki/Professor'},
 {'snippet': 'Feb 3, 2025 · Der Titel des Professors beziehungsweise der Professorin (kurz „Prof.“ beziehungsweise „Prof.in“) ist der höchste akademische Titel, der in der Wissenschaft verliehen …',
  'title': 'Professorentitel: Welche gibt es , wer darf sie führen? | academics',
  'link': 'https://www.academics.de/ratgeber/professoren-titel-titelfuehrung'},
 {'snippet': 'höchster akademischer Titel (der einem/einer [habilitierten] Hochschullehrer [in], verdienten Wissenschaftler [in], Künstler [in] o. Ä. verliehen wird) Grammatik ohne Plural Abkürzung Prof. …',
  'title': 'Professor Rechtschreibung, Bedeutung, Definition, Herkunft Duden',
  'link': 'https://www.duden.de/rechtschreibung/Professor'},
 {'snippet': 'Um dies 

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region="de-de")

search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format="list", num_results=5)

search.invoke("Prof Gregor Anderluh")

[{'snippet': 'January 30, 2025 - He taught at the University of Navarra in Pamplona (Spain), University of Vermont, and McGill University as a visiting professor.',
  'title': 'Wikipedia Greg N. Gregoriou - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Greg_N._Gregoriou'},
 {'snippet': 'September 20, 2025 - In 1960, he obtained employment as a philosophy instructor at Washington College, and in 1961 he received his doctorate at Columbia as an Irwin Edman Scholar and with Distinction in History after his dissertation on Giovanni Gentile. Gregor became assistant professor of philosophy at the University of Hawaii from 1961 to 1964.',
  'title': 'Wikipedia A. James Gregor - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/A._James_Gregor'},
 {'snippet': 'April 22, 2025 - Georges Anderla (March 27, 1921 in Prague - April 26, 2005 in Antibes) was a Czech-born French economist . While working for the OECD in 1973, he created a statistical model of the accumulation of human knowled

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

# 🧭 Richtige Region und Sprache
wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", safesearch="strict")

search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format="list", num_results=3)

# 🗝️ Suchbegriff präziser formulieren
query = "Kim A. Boutilier site:researchgate.net"

results = search.invoke(query)

for i, r in enumerate(results, start=1):
    print(f"🔹 Ergebnis {i}:")
    print(f"  Titel: {r['title']}")
    print(f"  Link: {r['link']}")
    print(f"  Beschreibung: {r['snippet']}\n")


🔹 Ergebnis 1:
  Titel: (PDF) Ectopic expression of BABY BOOM triggers a conversion from...
  Link: https://www.researchgate.net/publication/11215105_Ectopic_expression_of_BABY_BOOM_triggers_a_conversion_from_vegetative_to_embryonic_growth
  Beschreibung: from Vegetative to Embryonic Growth. Kim Boutilier , a,b,1.To whom correspondence should be addressed. E-mail k. a . boutilier @. plant.wag-ur.nl; fax 31-317-42-31-10.

🔹 Ergebnis 2:
  Titel: (PDF) Plasticity in Cell Division Patterns and Auxin Transport...
  Link: https://www.researchgate.net/publication/263273536_Plasticity_in_Cell_Division_Patterns_and_Auxin_Transport_Dependency_during_in_Vitro_Embryogenesis_in_Brassica_napus
  Beschreibung: Remko Offringa, e. and Kim Boutilier . a,1. a. Plant Research International, 6700 AP Wageningen, The Netherlands.

🔹 Ergebnis 3:
  Titel: (PDF) Auxin biosythesis maintains embryo identity and growth during...
  Link: https://www.researchgate.net/publication/356791425_Auxin_biosythesis_maintains_

In [14]:
import requests
from pprint import pprint

def fetch_openalex_id(name: str) -> dict | None:
    search_name = name.replace(" ", "+")
    base_url = f"https://api.openalex.org/authors?filter=display_name.search:{search_name}"
    response = requests.get(base_url)

    if response.status_code == 200:
        data = response.json()
        results_list = data.get('results', [])
        
        if not results_list:
            #st.warning(f"⚠️ Keine Ergebnisse für {name}")
            return None

        first_result = results_list[0]
        pprint(first_result)
        topics = first_result.get('topics', [])
        sorted_topics = sorted(topics, key=lambda x: x.get('count', 0), reverse=True)
        top_topics = [t['display_name'] for t in sorted_topics[:4]]

        affiliations = first_result.get('affiliations', [])
        
        # Funktion, um die aktuellste Institution zu finden
        def get_most_recent_affiliation(affiliations):
            if not affiliations:
                return None
            # Sortiere nach dem höchsten Jahr in der `years`-Liste
            most_recent = max(affiliations, key=lambda x: max(x.get('years', [0])))
            return most_recent['institution']

        # Aktuellste Institution abrufen
        most_recent_affiliation = get_most_recent_affiliation(affiliations)
        x_concepts = first_result.get('x_concepts', [])
        x_concepts_fields_list = [
            concept['display_name'] for concept in x_concepts if concept.get("score", 0) >= 90
        ]

        return {
            "Name": name,
            "topics": top_topics,
            "affiliation": most_recent_affiliation['display_name'] if most_recent_affiliation else None,
            "x_concepts": x_concepts_fields_list
        }

    else:
        #st.error(f"⚠️ Fehler bei Anfrage: {response.status_code}")
        return None
    
fetch_openalex_id("Gregor Anderluh")

{'affiliations': [{'institution': {'country_code': 'US',
                                   'display_name': 'Google (United States)',
                                   'id': 'https://openalex.org/I1291425158',
                                   'lineage': ['https://openalex.org/I1291425158',
                                               'https://openalex.org/I4210128969'],
                                   'ror': 'https://ror.org/00njsd438',
                                   'type': 'company'},
                   'years': [2017]},
                  {'institution': {'country_code': 'SI',
                                   'display_name': 'University of Ljubljana',
                                   'id': 'https://openalex.org/I153976015',
                                   'lineage': ['https://openalex.org/I153976015'],
                                   'ror': 'https://ror.org/05njb9z20',
                                   'type': 'education'},
                   'years': [2024,
  

{'Name': 'Gregor Anderluh',
 'topics': ['Lipid Membrane Structure and Behavior',
  'Marine Invertebrate Physiology and Ecology',
  'RNA and protein synthesis mechanisms',
  'Mycotoxins in Agriculture and Food'],
 'affiliation': 'National Institute of Chemistry',
 'x_concepts': []}

In [1]:
import requests
from bs4 import BeautifulSoup
import time

def fetch_researchgate_profile(name: str) -> str | None:
    """
    Holt den HTML-Code einer ResearchGate-Profilseite.
    
    Args:
        name: Name im Format "Vorname-Nachname" (z.B. "Gregor-Anderluh")
    
    Returns:
        HTML-Code als String oder None bei Fehler
    """
    url = f"https://www.researchgate.net/profile/{name}"
    
    # Wichtig: User-Agent setzen, um wie ein Browser auszusehen
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.researchgate.net/',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        # HTML parsen
        soup = BeautifulSoup(response.text, 'html.parser')
        
        return response.text
    
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Fehler beim Abrufen von {url}: {e}")
        return None

# Beispiel
html = fetch_researchgate_profile("Gregor-Anderluh")
if html:
    print(html[:500])  # Erste 500 Zeichen anzeigen

⚠️ Fehler beim Abrufen von https://www.researchgate.net/profile/Gregor-Anderluh: 403 Client Error: Forbidden for url: https://www.researchgate.net/profile/Gregor-Anderluh


In [ ]:
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="origin" name="referrer"/>
  <meta content="aad-pbMLxv3ZQZZ0HrPM9VUG8hyhsbWxXOoJEZJvSB9HUoub0UsRxcCvObWZA3EXhGMV+9ib42XFVX6AW/bJcNKuCEfWfDeuPwrYCWZemmZfvzX5xxvfEfERpBRwrYLm8mmAVZ/qzgYY74D6DsyVjQ9pHZmwh7ToHJJzzCP/kbkM4C22ATtHlDvb/q1qv1kKu/s/LrDdmZkDsWgC1o4lJFSqqEGTWmHdiDWayPhSAqMfd8bzWgBrJLYeKV31YOaIQRXvXwGtcLT3zGrCDGUPu3E=" id="Rg-Request-Token" name="Rg-Reque

<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="origin" name="referrer"/>
  <meta content="aad-pbMLxv3ZQZZ0HrPM9VUG8hyhsbWxXOoJEZJvSB9HUoub0UsRxcCvObWZA3EXhGMV+9ib42XFVX6AW/bJcNKuCEfWfDeuPwrYCWZemmZfvzX5xxvfEfERpBRwrYLm8mmAVZ/qzgYY74D6DsyVjQ9pHZmwh7ToHJJzzCP/kbkM4C22ATtHlDvb/q1qv1kKu/s/LrDdmZkDsWgC1o4lJFSqqEGTWmHdiDWayPhSAqMfd8bzWgBrJLYeKV31YOaIQRXvXwGtcLT3zGrCDGUPu3E=" id="Rg-Request-Token" name="Rg-Reque


In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random

def fetch_researchgate_advanced(name: str) -> dict | None:
    """
    Erweiterte Methode mit besserer Bot-Erkennung-Umgehung.
    
    Installation:
        pip install selenium undetected-chromedriver
        brew install chromedriver (Mac)
    """
    url = f"https://www.researchgate.net/profile/{name}"
    
    chrome_options = Options()
    
    # Wichtig: NICHT headless verwenden (wird oft erkannt)
    # chrome_options.add_argument('--headless')
    
    # Anti-Detection-Argumente
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--no-sandbox')
    
    # Realistischer User-Agent
    chrome_options.add_argument(
        'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/120.0.0.0 Safari/537.36'
    )
    
    try:
        driver = webdriver.Chrome(options=chrome_options)
        
        # WebDriver-Property überschreiben (Anti-Detection)
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        # Seite laden
        driver.get(url)
        
        # Zufällige Wartezeit (menschlicher)
        time.sleep(random.uniform(3, 5))
        
        # Warten auf spezifisches Element
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "h1"))
            )
        except:
            print("⚠️ Timeout beim Laden der Seite")
            driver.quit()
            return None
        
        # HTML abrufen und parsen
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Informationen extrahieren
        result = extract_info_from_soup(soup)
        
        driver.quit()
        return result
    
    except Exception as e:
        print(f"⚠️ Fehler: {e}")
        try:
            driver.quit()
        except:
            pass
        return None


def extract_info_from_soup(soup: BeautifulSoup) -> dict:
    """Extrahiert Informationen aus BeautifulSoup-Objekt."""
    
    # Name
    name_tag = soup.find('h1', {'class': lambda x: x and 'nova' in x})
    name = name_tag.text.strip() if name_tag else None
    
    # Institution (mehrere mögliche Klassen)
    institution = None
    institution_selectors = [
        {'class': 'nova-legacy-v-institution-item__title'},
        {'class': lambda x: x and 'institution' in str(x).lower()},
    ]
    
    for selector in institution_selectors:
        institution_tag = soup.find('div', selector)
        if institution_tag:
            institution = institution_tag.text.strip()
            break
    
    # Forschungsinteressen
    interests = []
    interest_tags = soup.find_all('a', {'class': lambda x: x and 'skill' in str(x).lower()})
    interests = [tag.text.strip() for tag in interest_tags[:5]]
    
    # Abteilung/Department
    dept_tag = soup.find('div', {'class': lambda x: x and 'department' in str(x).lower()})
    department = dept_tag.text.strip() if dept_tag else None
    
    return {
        "name": name,
        "institution": institution,
        "department": department,
        "interests": interests
    }

In [18]:
from bs4 import BeautifulSoup
import requests

def extract_researchgate_info(name: str) -> dict | None:
    """Extrahiert wichtige Informationen vom ResearchGate-Profil."""
    url = f"https://www.researchgate.net/profile/{name}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Name extrahieren
        name_tag = soup.find('h1', class_='nova-legacy-e-text')
        researcher_name = name_tag.text.strip() if name_tag else None
        
        # Institution extrahieren
        institution_tag = soup.find('div', class_='nova-legacy-v-institution-item__title')
        institution = institution_tag.text.strip() if institution_tag else None
        
        # Forschungsinteressen
        interests = []
        interest_tags = soup.find_all('a', class_='nova-legacy-e-link--theme-bare')
        for tag in interest_tags[:5]:  # Nur die ersten 5
            interests.append(tag.text.strip())
        
        return {
            "name": researcher_name,
            "institution": institution,
            "interests": interests,
            "profile_url": url
        }
    
    except Exception as e:
        print(f"⚠️ Fehler: {e}")
        return None

# Beispiel
info = extract_researchgate_info("Gregor-Anderluh")
if info:
    from pprint import pprint
    pprint(info)

⚠️ Fehler: 403 Client Error: Forbidden for url: https://www.researchgate.net/profile/Gregor-Anderluh


In [19]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time

def fetch_with_undetected_chrome(name: str) -> dict | None:
    """
    Verwendet undetected-chromedriver (schwerer zu erkennen).
    
    Installation:
        pip install undetected-chromedriver
    """
    url = f"https://www.researchgate.net/profile/{name}"
    
    try:
        # Undetected ChromeDriver initialisieren
        options = uc.ChromeOptions()
        options.add_argument('--disable-blink-features=AutomationControlled')
        
        driver = uc.Chrome(options=options)
        driver.get(url)
        
        # Warten und scrollen (menschliches Verhalten simulieren)
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 500)")
        time.sleep(2)
        
        # HTML abrufen
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        result = extract_info_from_soup(soup)
        
        driver.quit()
        return result
    
    except Exception as e:
        print(f"⚠️ Fehler: {e}")
        return None

In [21]:
import time
import random
from functools import wraps

def rate_limited(min_delay=2, max_delay=5):
    """Decorator für Rate-Limiting."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            time.sleep(random.uniform(min_delay, max_delay))
            return func(*args, **kwargs)
        return wrapper
    return decorator

@rate_limited(min_delay=3, max_delay=6)
def fetch_profile(name):
    return fetch_with_undetected_chrome(name)

info = fetch_profile("Gregor-Anderluh")

⚠️ Fehler: Binary Location Must be a String


In [24]:
def get_researcher_info(name: str) -> dict | None:
    """
    Hybrid-Ansatz: OpenAlex → Undetected Chrome → Fehler
    """
    print(f"🔍 Suche Informationen für: {name}")
    
    # 1. Versuch: OpenAlex (schnell & zuverlässig)
    # openalex_data = fetch_openalex_id(name)
    # if openalex_data and openalex_data.get('affiliation'):
    #     print("✅ OpenAlex erfolgreich")
    #     return openalex_data
    
    print("⚠️ OpenAlex unvollständig, versuche ResearchGate...")
    
    # 2. Versuch: ResearchGate mit Undetected Chrome
    time.sleep(random.uniform(2, 4))  # Rate-Limiting
    
    rg_name = name.replace(" ", "-")  # Format für ResearchGate
    rg_data = fetch_with_undetected_chrome(rg_name)
    
    if rg_data and rg_data.get('institution'):
        print("✅ ResearchGate erfolgreich")
        # OpenAlex-Daten mit ResearchGate ergänzen
        # if openalex_data:
        #     openalex_data['affiliation'] = rg_data['institution']
        #     return openalex_data
        return rg_data
    
    print("❌ Keine Daten gefunden")
    return None  # Gib zurück, was du hast


# Beispiel-Nutzung
names = ["Gregor Anderluh", "Kim A. Boutilier"]

for name in names:
    result = get_researcher_info(name)
    if result:
        print(f"\n📊 Ergebnis für {name}:")
        pprint(result)
    time.sleep(random.uniform(3, 6))  # Pause zwischen Anfragen

🔍 Suche Informationen für: Gregor Anderluh
⚠️ OpenAlex unvollständig, versuche ResearchGate...
⚠️ Fehler: Binary Location Must be a String
❌ Keine Daten gefunden
🔍 Suche Informationen für: Kim A. Boutilier
⚠️ OpenAlex unvollständig, versuche ResearchGate...
⚠️ Fehler: Binary Location Must be a String
❌ Keine Daten gefunden


In [3]:
import pandas as pd
from pprint import pprint
import requests

def fetch_openalex_id(name: str) -> dict | None:
    search_name = name.replace(" ", "+")
    base_url = f"https://api.openalex.org/authors?filter=display_name.search:{search_name}"
    response = requests.get(base_url)

    if response.status_code == 200:
        data = response.json()
        results_list = data.get('results', [])
        
        if not results_list:
            #st.warning(f"⚠️ Keine Ergebnisse für {name}")
            return None

        first_result = results_list[0]
        #pprint(first_result)
        topics = first_result.get('topics', [])
        sorted_topics = sorted(topics, key=lambda x: x.get('count', 0), reverse=True)
        top_topics = [t['display_name'] for t in sorted_topics[:4]]
        orcid = first_result.get('orcid', None)

        affiliations = first_result.get('affiliations', [])
        
        # Funktion, um die aktuellste Institution zu finden
        def get_most_recent_affiliation(affiliations):
            if not affiliations:
                return None
            # Sortiere nach dem höchsten Jahr in der `years`-Liste
            most_recent = max(affiliations, key=lambda x: max(x.get('years', [0])))
            return most_recent['institution']

        # Aktuellste Institution abrufen
        most_recent_affiliation = get_most_recent_affiliation(affiliations)
        x_concepts = first_result.get('x_concepts', [])
        x_concepts_fields_list = [
            concept['display_name'] for concept in x_concepts if concept.get("score", 0) >= 90
        ]

        return {
            "Name": name,
            "topics": top_topics,
            "affiliation": most_recent_affiliation['display_name'] if most_recent_affiliation else None,
            "x_concepts": x_concepts_fields_list,
            "orcid": orcid
        }

    else:
        #st.error(f"⚠️ Fehler bei Anfrage: {response.status_code}")
        return None
    
data = fetch_openalex_id("Gregor Anderluh")

In [22]:
import requests
import json

def get_orcid_data(orcid_id):
    """
    Ruft öffentliche Daten von ORCID ab
    
    Args:
        orcid_id: ORCID ID (z.B. '0000-0002-9916-8465')
    """
    base_url = "https://pub.orcid.org/v3.0/"
    headers = {
        'Accept': 'application/json'
    }
    
    # Vollständiges Profil abrufen
    url = f"{base_url}{orcid_id}"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Fehler: {response.status_code}")
        return None

# Beispiel für deine ORCID
orcid_id = "0000-0002-9916-8465"
data = get_orcid_data(orcid_id)

if data:
    # Grundlegende Informationen extrahieren
    person = data.get('person', {})
    name = person.get('name', {})
    
    pprint(data)

{'activities-summary': {'distinctions': {'affiliation-group': [],
                                         'last-modified-date': None,
                                         'path': '/0000-0002-9916-8465/distinctions'},
                        'educations': {'affiliation-group': [{'external-ids': {'external-id': []},
                                                              'last-modified-date': {'value': 1405865258524},
                                                              'summaries': [{'education-summary': {'created-date': {'value': 1393672134137},
                                                                                                   'department-name': 'Biotechnical '
                                                                                                                      'Faculty',
                                                                                                   'display-index': '0',
                                            

In [20]:
def get_orcid_works(orcid_id):
    """Ruft nur Publikationen ab"""
    url = f"https://pub.orcid.org/v3.0/{orcid_id}/works"
    headers = {'Accept': 'application/json'}
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        works = response.json()
        return works.get('group', [])
    return None

# Publikationen abrufen
works = get_orcid_works("0000-0002-9916-8465")
if works:
    print(f"\nAnzahl Publikationen: {len(works)}")
    for work in works[:100]:  # Erste 3 anzeigen
        summary = work.get('work-summary', [{}])[0]
        title = summary.get('title', {}).get('title', {}).get('value')
        year = summary.get('publication-date', {}).get('year', {}).get('value')
        print(f"- {title} ({year})")


Anzahl Publikationen: 143
- Plasticity of the cytotoxic Nep1-like protein enables promiscuity in binding to its lipid receptor glycosylinositol phosphorylceramides (2025)
- Interaction of an Oomycete Nep1-like Cytolysin with Natural and Plant Cell-Mimicking Membranes (2025)
- The Biological Role of Conoporins, Actinoporin-like Pore-Forming Toxins from Cone Snails (2025)
- Cryo-EM structures of a protein pore reveal a cluster of cholesterol molecules and diverse roles of membrane lipids (2025)
- High-resolution cryo-EM structures of a protein pore reveal diverse roles of membrane lipids (2024)
- Web of venom: exploration of big data resources in animal toxin research (2024)
- Programmable de novo designed coiled coil-mediated phase separation in mammalian cells (2023)
- Structural basis for the unique molecular properties of broad-range phospholipase C from Listeria monocytogenes (2023)
- Bacillus thuringiensis toxin Cyt2Aa forms filamentous oligomers when exposed to lipid membranes or

In [23]:
import requests

def get_orcid_keywords(orcid_id):
    """Ruft Keywords/Research Topics ab"""
    url = f"https://pub.orcid.org/v3.0/{orcid_id}/keywords"
    headers = {'Accept': 'application/json'}
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        keywords = data.get('keyword', [])
        
        print("Research Keywords/Topics:")
        for kw in keywords:
            content = kw.get('content')
            print(f"  - {content}")
        
        return keywords
    return None

# Beispiel
orcid_id = "0000-0002-9916-8465"
keywords = get_orcid_keywords(orcid_id)

Research Keywords/Topics:
  - proteins, lipid membranes, protein-membrane interactions, pore forming proteins, toxins


In [26]:
def get_research_resources(orcid_id):
    """Ruft Research Resources ab"""
    url = f"https://pub.orcid.org/v3.0/{orcid_id}/research-resources"
    headers = {'Accept': 'application/json'}
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return None

# Beispiel
orcid_id = "0000-0002-9916-8465"
resources = get_research_resources(orcid_id)
if resources:
    pprint(resources)

{'group': [],
 'last-modified-date': None,
 'path': '/0000-0002-9916-8465/research-resources'}


In [28]:
class ORCIDClient:
    """Client für ORCID API."""
    
    BASE_URL = "https://pub.orcid.org/v3.0"
    
    @staticmethod
    def get_profile(orcid_id: str) -> dict | None:
        """Holt vollständiges ORCID-Profil."""
        url = f"{ORCIDClient.BASE_URL}/{orcid_id}"
        headers = {'Accept': 'application/json'}
        
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"❌ ORCID API Fehler: {e}")
            return None
    
    @staticmethod
    def get_keywords(orcid_id: str) -> list[str]:
        """Holt nur Keywords."""
        url = f"{ORCIDClient.BASE_URL}/{orcid_id}/keywords"
        headers = {'Accept': 'application/json'}
        
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            keywords_list = data.get('keyword', [])
            return [
                kw.get('content') 
                for kw in keywords_list 
                if kw.get('content')
            ]
        except:
            return []
    
    @staticmethod
    def get_biography(orcid_id: str) -> str | None:
        """Holt nur Biografie."""
        profile = ORCIDClient.get_profile(orcid_id)
        
        if not profile:
            return None
        
        try:
            person = profile.get('person', {})
            biography_obj = person.get('biography')
            
            if biography_obj and isinstance(biography_obj, dict):
                return biography_obj.get('content')
        except:
            pass
        
        return None
    
    @staticmethod
    def get_research_info(orcid_id: str) -> dict:
        """
        Kombiniert alle relevanten Forschungsinformationen.
        
        Returns:
            Dictionary mit biography, keywords, works_count
        """
        profile = ORCIDClient.get_profile(orcid_id)
        
        if not profile:
            return {
                'biography': None,
                'keywords': [],
                'works_count': 0,
                'orcid': orcid_id
            }
        
        # Biografie
        biography = None
        person = profile.get('person', {})
        bio_obj = person.get('biography')
        if bio_obj and isinstance(bio_obj, dict):
            biography = bio_obj.get('content')
        
        # Keywords
        keywords = []
        kw_obj = person.get('keywords', {})
        if kw_obj and isinstance(kw_obj, dict):
            kw_list = kw_obj.get('keyword', [])
            keywords = [
                kw.get('content') 
                for kw in kw_list 
                if isinstance(kw, dict) and kw.get('content')
            ]
        
        # Publikationsanzahl
        works_count = 0
        activities = profile.get('activities-summary', {})
        if activities:
            works = activities.get('works', {})
            if works and isinstance(works, dict):
                works_group = works.get('group', [])
                works_count = len(works_group) if isinstance(works_group, list) else 0
        
        return {
            'biography': biography,
            'keywords': keywords,
            'works_count': works_count,
            'orcid': orcid_id
        }


# Beispiel-Nutzung
orcid_id = "0000-0002-9916-8465"

# Einzelne Funktionen
print("📖 Biografie:")
print(ORCIDClient.get_biography(orcid_id))

print("\n🔑 Keywords:")
print(ORCIDClient.get_keywords(orcid_id))

# Vollständige Info
print("\n📊 Vollständige Forschungsinfo:")
info = ORCIDClient.get_research_info(orcid_id)
pprint(info)

📖 Biografie:
None

🔑 Keywords:
['proteins, lipid membranes, protein-membrane interactions, pore forming proteins, toxins']

📊 Vollständige Forschungsinfo:
{'biography': None,
 'keywords': ['proteins, lipid membranes, protein-membrane interactions, pore '
              'forming proteins, toxins'],
 'orcid': '0000-0002-9916-8465',
 'works_count': 143}


In [37]:
class ORCIDClient:
    """Client für ORCID API."""
    
    BASE_URL = "https://pub.orcid.org/v3.0"
    
    @staticmethod
    def get_profile(orcid_id: str) -> dict | None:
        """Holt vollständiges ORCID-Profil."""
        url = f"{ORCIDClient.BASE_URL}/{orcid_id}"
        headers = {'Accept': 'application/json'}
        
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"❌ ORCID API Fehler: {e}")
            return None
    
    @staticmethod
    def get_current_affiliation(orcid_id: str) -> str | None:
        """
        Holt die aktuelle Affiliation (neueste Employment oder Education).
        
        Returns:
            Name der Institution oder None
        """
        profile = ORCIDClient.get_profile(orcid_id)
        
        if not profile:
            return None
        
        try:
            activities = profile.get('activities-summary', {})
            
            # 1. Versuch: Employments (bevorzugt)
            employments = activities.get('employments', {})
            if employments:
                employment_groups = employments.get('affiliation-group', [])
                
                # Alle Employments mit End-Datum = None (aktuell) oder neuestes Jahr
                current_employments = []
                
                for group in employment_groups:
                    summaries = group.get('summaries', [])
                    for summary in summaries:
                        employment_summary = summary.get('employment-summary', {})
                        
                        # Prüfe ob aktuell (kein End-Datum)
                        end_date = employment_summary.get('end-date')
                        
                        if end_date is None:  # Aktuell
                            org_name = employment_summary.get('organization', {}).get('name')
                            if org_name:
                                return org_name
                        
                        # Sammle alle mit Datum für Fallback
                        start_date = employment_summary.get('start-date', {})
                        year = start_date.get('year', {}).get('value', 0) if start_date else 0
                        
                        current_employments.append({
                            'name': employment_summary.get('organization', {}).get('name'),
                            'year': year,
                            'end_date': end_date
                        })
                
                # Wenn kein aktuelles gefunden: Nimm das neueste
                if current_employments:
                    latest = max(current_employments, key=lambda x: x['year'])
                    return latest['name']
            
            # # 2. Versuch: Education (falls keine Employment)
            # educations = activities.get('educations', {})
            # if educations:
            #     education_groups = educations.get('affiliation-group', [])
                
            #     for group in education_groups:
            #         summaries = group.get('summaries', [])
            #         for summary in summaries:
            #             education_summary = summary.get('education-summary', {})
                        
            #             end_date = education_summary.get('end-date')
            #             if end_date is None:  # Noch in Ausbildung
            #                 org_name = education_summary.get('organization', {}).get('name')
            #                 if org_name:
            #                     return org_name
            
            # # 3. Fallback: Erste gefundene Institution
            # if employment_groups:
            #     first_group = employment_groups[0]
            #     summaries = first_group.get('summaries', [])
            #     if summaries:
            #         first_summary = summaries[0].get('employment-summary', {})
            #         org = first_summary.get('organization', {})
            #         return org.get('name')
        
        except (KeyError, ValueError, TypeError) as e:
            print(f"⚠️ Fehler beim Extrahieren der Affiliation: {e}")
        
        return None
    
    @staticmethod
    def get_keywords(orcid_id: str) -> list[str]:
        """Holt nur Keywords."""
        url = f"{ORCIDClient.BASE_URL}/{orcid_id}/keywords"
        headers = {'Accept': 'application/json'}
        
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            keywords_list = data.get('keyword', [])
            return [
                kw.get('content') 
                for kw in keywords_list 
                if kw.get('content')
            ]
        except:
            return []
    
    @staticmethod
    def get_biography(orcid_id: str) -> str | None:
        """Holt nur Biografie."""
        profile = ORCIDClient.get_profile(orcid_id)
        
        if not profile:
            return None
        
        try:
            person = profile.get('person', {})
            biography_obj = person.get('biography')
            
            if biography_obj and isinstance(biography_obj, dict):
                return biography_obj.get('content')
        except:
            pass
        
        return None
    
    @staticmethod
    def get_research_info(orcid_id: str) -> dict:
        """
        Kombiniert alle relevanten Forschungsinformationen inkl. Affiliation.
        
        Returns:
            Dictionary mit affiliation, biography, keywords, works_count
        """
        profile = ORCIDClient.get_profile(orcid_id)
        
        if not profile:
            return {
                'affiliation': None,
                'biography': None,
                'keywords': [],
                'works_count': 0,
                'orcid': orcid_id
            }
        
        # Aktuelle Affiliation
        affiliation = ORCIDClient.get_current_affiliation(orcid_id)
        
        # Biografie
        biography = None
        person = profile.get('person', {})
        bio_obj = person.get('biography')
        if bio_obj and isinstance(bio_obj, dict):
            biography = bio_obj.get('content')
        
        # Keywords
        keywords = []
        kw_obj = person.get('keywords', {})
        if kw_obj and isinstance(kw_obj, dict):
            kw_list = kw_obj.get('keyword', [])
            keywords = [
                kw.get('content') 
                for kw in kw_list 
                if isinstance(kw, dict) and kw.get('content')
            ]
        
        # Publikationsanzahl
        works_count = 0
        activities = profile.get('activities-summary', {})
        if activities:
            works = activities.get('works', {})
            if works and isinstance(works, dict):
                works_group = works.get('group', [])
                works_count = len(works_group) if isinstance(works_group, list) else 0
        
        return {
            'affiliation': affiliation,
            'biography': biography,
            'keywords': keywords,
            'works_count': works_count,
            'orcid': orcid_id
        }


# Test
orcid_id = "0000-0002-9916-8465"

print("🏢 Aktuelle Affiliation:")
print(ORCIDClient.get_current_affiliation(orcid_id))

print("\n📊 Vollständige Info:")
info = ORCIDClient.get_research_info(orcid_id)
pprint(info)

🏢 Aktuelle Affiliation:
National Institute of Chemistry

📊 Vollständige Info:
{'affiliation': 'National Institute of Chemistry',
 'biography': None,
 'keywords': ['proteins, lipid membranes, protein-membrane interactions, pore '
              'forming proteins, toxins'],
 'orcid': '0000-0002-9916-8465',
 'works_count': 143}


In [38]:
def fetch_openalex_orcid_only(name: str) -> str | None:
    """
    Holt nur die ORCID-ID von OpenAlex.
    
    Args:
        name: Vollständiger Name des Forschers
        
    Returns:
        ORCID-ID als String oder None
    """
    search_name = name.replace(" ", "+")
    base_url = f"https://api.openalex.org/authors?filter=display_name.search:{search_name}"
    
    try:
        response = requests.get(base_url, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        results_list = data.get('results', [])
        
        if not results_list:
            print(f"⚠️ Keine OpenAlex-Ergebnisse für '{name}'")
            return None
        
        first_result = results_list[0]
        orcid = first_result.get('orcid')
        
        if orcid:
            # ORCID ID extrahieren (URL → ID)
            orcid_id = orcid.split('/')[-1] if '/' in orcid else orcid
            return orcid_id
        
        print(f"⚠️ Keine ORCID gefunden für '{name}'")
        return None
    
    except Exception as e:
        print(f"❌ Fehler für '{name}': {e}")
        return None


def fetch_researcher_info_orcid_first(name: str) -> dict | None:
    """
    Holt ORCID von OpenAlex und alle Daten (inkl. Affiliation) von ORCID.
    
    Args:
        name: Vollständiger Name des Forschers
        
    Returns:
        Dictionary mit allen Forscherdaten oder None
    """
    print(f"🔍 Suche ORCID für: {name}")
    
    # 1. ORCID von OpenAlex holen
    orcid_id = fetch_openalex_orcid_only(name)
    
    if not orcid_id:
        print(f"❌ Keine ORCID gefunden für '{name}'")
        return None
    
    print(f"✅ ORCID gefunden: {orcid_id}")
    
    # 2. Alle Daten von ORCID holen
    orcid_data = ORCIDClient.get_research_info(orcid_id)
    
    if not orcid_data:
        print(f"⚠️ ORCID-Daten konnten nicht abgerufen werden")
        return None
    
    # 3. Ergebnis zusammenstellen
    return {
        "name": name,
        "orcid": orcid_id,
        "affiliation": orcid_data.get('affiliation'),
        "topics": orcid_data.get('keywords', [])[:10],  # Max 6 Keywords
        "biography": orcid_data.get('biography'),
        "works_count": orcid_data.get('works_count', 0)
    }


# Test
data = fetch_researcher_info_orcid_first("Gregor Anderluh")
if data:
    print("\n📊 Ergebnis:")
    pprint(data)

🔍 Suche ORCID für: Gregor Anderluh
✅ ORCID gefunden: 0000-0002-9916-8465

📊 Ergebnis:
{'affiliation': 'National Institute of Chemistry',
 'biography': None,
 'name': 'Gregor Anderluh',
 'orcid': '0000-0002-9916-8465',
 'topics': ['proteins, lipid membranes, protein-membrane interactions, pore '
            'forming proteins, toxins'],
 'works_count': 143}
